# 🏠 **Projet Data Science – Outil d’Aide à la Décision pour Investisseurs Immobiliers**

---

### 👥 **Équipe Projet**

**Classe :** ECE – Data Science  

**Membres :**  
- Alan LATEB  
- Bassel ALEIAN  
- Jean-Charles MCHANGAMA
- Ayoub Fetah

---

---

## 🧩 **I. Persona –Marc DUBOIS**

### 👤 **Profil**

**Nom :** Marc DUBOIS
**Âge :** 42 ans  
**Profession :** Chef d’entreprise dans le bâtiment (rénovation)  
**Localisation :** Région Parisienne (Essonne)  
**Expérience :** 12 projets achat–rénovation–revente réalisés  
**Capital disponible :** 80 000 € à 150 000 € par projet  
**Stratégie :** Achète 2–3 biens/an, rénove en 3–6 mois, revend avec une marge de 20–30 %

---

### 🎯 **Besoins Principaux**

1. Identifier les biens **sous-évalués** par rapport au marché  
2. Évaluer le **potentiel de plus-value** après rénovation  
3. Repérer les **zones en gentrification** ou valorisation  
4. Trouver les **biens anciens avec bon ratio surface/prix**  
5. Analyser l’**écart de prix entre ancien et neuf** par secteur  
6. Évaluer la **liquidité du marché local**  
7. Identifier les **biens éligibles aux aides à la rénovation**  
8. Détecter les **biens avec terrain exploitable**  
9. Comparer plusieurs **communes selon critères multiples**  
10. Obtenir un **classement automatique** des meilleures opportunités d’investissement

---


### Imports nécessaires

In [10]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import ipywidgets as widgets
from IPython.display import display, clear_output
import webbrowser
import os
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### Chargement et nettoyage global du fichier DVF

Dans cette cellule, on charge le fichier **`data/dvf.csv`** (France entière)  
et on effectue un premier nettoyage :
- Filtrage sur les **ventes** de **maisons et appartements** en **Île-de-France**  
- Suppression des valeurs aberrantes  
- Calcul du **prix au m²** et extraction de l’**année de mutation**

Cette base propre servira ensuite à créer les fichiers spécifiques pour les widgets.


In [ ]:
"""
# ===============================================================
# 🧹 Étape 1 : Chargement et nettoyage global du fichier DVF
# ---------------------------------------------------------------
# Objectif : Créer une base propre (ventes, maisons/appartements, IDF)
#             qui servira aux extractions suivantes.
# Fichier source : data/dvf.csv
# ===============================================================

import pandas as pd
import numpy as np

# --- Lecture du fichier brut DVF ---
cols = [
    'date_mutation', 'nature_mutation', 'valeur_fonciere',
    'surface_reelle_bati', 'type_local', 'code_postal',
    'code_commune', 'code_departement'
]

print("⏳ Chargement du fichier DVF brut...")
df = pd.read_csv("data/dvf.csv", usecols=cols, low_memory=False)
print(f"Fichier brut chargé : {df.shape[0]:,} lignes")

# --- Nettoyage de base ---
df['valeur_fonciere'] = pd.to_numeric(df['valeur_fonciere'], errors='coerce')
df['surface_reelle_bati'] = pd.to_numeric(df['surface_reelle_bati'], errors='coerce')
df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')

# Codes départements Île-de-France
deps_idf = ['75','77','78','91','92','93','94','95']
df['code_departement'] = df['code_departement'].astype(str).str.zfill(2)

# --- Filtrage ---
mask = (
    (df['nature_mutation'] == 'Vente') &
    (df['type_local'].isin(['Maison', 'Appartement'])) &
    (df['surface_reelle_bati'] > 10) &
    (df['valeur_fonciere'] > 1000) &
    (df['code_departement'].isin(deps_idf))
)
df = df.loc[mask].copy()

print(f"✅ Après filtrage : {df.shape[0]:,} ventes en IDF (maisons/appartements)")

# --- Calcul du prix au m² ---
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

# Suppression des extrêmes (1 % les plus bas et les plus hauts)
q1, q99 = df['prix_m2'].quantile([0.01, 0.99])
df = df[(df['prix_m2'] >= q1) & (df['prix_m2'] <= q99)]

# --- Ajout de l'année ---
df['annee'] = df['date_mutation'].dt.year

print("✅ Base DVF propre prête à être utilisée pour les extractions.")
"""

### Extraction pour le Widget 1 (2022–2024)

On extrait uniquement les **ventes récentes (2022–2024)**  
et les colonnes nécessaires pour le **Widget 1 : Distribution du prix au m²**.  
Ce fichier sera plus léger et représentatif du **marché actuel**.


In [ ]:
"""
# ===============================================================
# 📁 Étape 2 : Création du fichier pour le Widget 1
# ---------------------------------------------------------------
# Widget 1 : Distribution du prix au m²
# Période : 2022–2024
# Objectif : Visualiser la répartition des prix au m²
# ===============================================================

df_22_24 = df[(df['annee'] >= 2022) & (df['annee'] <= 2024)].copy()

cols_widget1 = [
    'date_mutation', 'annee', 'valeur_fonciere',
    'surface_reelle_bati', 'prix_m2',
    'type_local', 'code_postal', 'code_departement'
]
df_22_24 = df_22_24[cols_widget1]

# Sauvegarde
output_path_1 = "data/dvf_idf_2022_2024_clean.csv"
df_22_24.to_csv(output_path_1, index=False)

print(f"📦 Fichier Widget 1 sauvegardé : {output_path_1}")
print(f"➡️ Lignes : {df_22_24.shape[0]:,} | Colonnes : {len(df_22_24.columns)}")
"""

### Extraction pour le Widget 3 (2019–2024)

On extrait les **ventes entre 2019 et 2024** pour analyser  
l’**évolution temporelle du prix au m²** dans le **Widget 3**.  
Cette période de 5 ans permet de visualiser les **tendances du marché**  
et de repérer les **zones en gentrification**.


In [ ]:
"""
# ===============================================================
# 📁 Étape 3 : Création du fichier pour le Widget 3
# ---------------------------------------------------------------
# Widget 3 : Évolution temporelle du prix au m²
# Période : 2019–2024
# Objectif : Analyser les tendances de prix par commune
# ===============================================================

df_19_24 = df[(df['annee'] >= 2019) & (df['annee'] <= 2024)].copy()

cols_widget3 = [
    'date_mutation', 'annee', 'valeur_fonciere',
    'surface_reelle_bati', 'prix_m2',
    'type_local', 'code_commune', 'code_departement'
]
df_19_24 = df_19_24[cols_widget3]

# Sauvegarde
output_path_2 = "data/dvf_idf_2019_2024_clean.csv"
df_19_24.to_csv(output_path_2, index=False)

print(f"📦 Fichier Widget 3 sauvegardé : {output_path_2}")
print(f"➡️ Lignes : {df_19_24.shape[0]:,} | Colonnes : {len(df_19_24.columns)}")
print("\n✅ Nettoyage terminé avec succès !")
"""

---

## 🧩 Widget 1 — Distribution du prix au m² (2022 – 2024)

### 🎯 Objectif
Ce widget permet à **Marc Dubois**, investisseur en réhabilitation immobilière,  
de **visualiser la répartition du prix au m²** sur le marché actuel de l’Île-de-France.  
L’objectif est d’identifier rapidement les **zones accessibles** et les **biens potentiellement sous-évalués**.

---

### ⚙️ Données utilisées
Fichier : `data/dvf_idf_2022_2024_clean.csv`  
Colonnes exploitées :
- `annee` — Année de mutation  
- `code_departement` — Département du bien  
- `type_local` — Type de bien (Maison / Appartement)  
- `prix_m2` — Prix de vente au m² calculé

---

### 🧰 Paramètres interactifs
| Paramètre | Type | Description |
|------------|------|-------------|
| **Département(s)** | Sélecteur multiple | Filtrer un ou plusieurs départements d’Île-de-France |
| **Type de bien** | Sélecteur multiple | Choisir entre *Maisons* et/ou *Appartements* |
| **Année** | Slider | Sélectionner l’année de transaction (2022–2024) |
| **Échelle logarithmique** | Checkbox | Lisse l’effet des valeurs extrêmes sur l’histogramme |

---

### 📊 Sortie du widget
- **Histogramme interactif** du prix au m² selon les filtres sélectionnés  
  - Couleurs et style adaptés au thème du notebook (*clair ou sombre*)  
  - Ligne verticale indiquant la **médiane**  
- **Statistiques clés** calculées dynamiquement :
  - Nombre de ventes  
  - Médiane du prix au m²  
  - Quartiles (Q1 – Q3)

---

### 🧠 Interprétation attendue
Ce graphique aide à :
- Comprendre la **répartition des ventes** selon les gammes de prix  
- Repérer les **zones abordables** par rapport à la moyenne régionale  
- Estimer la **capacité d’achat** de Marc en fonction de son budget (80–150 k€)  
- Servir de **base comparative** avant d’approfondir par commune

---

### 💶 Ordres de grandeur observés (réalité 2024-2025)
| Département | Code | Prix moyen €/m² |
|--------------|------|----------------|
| Paris | 75 | ~9 700 € |
| Hauts-de-Seine | 92 | ~6 600 € |
| Val-de-Marne | 94 | ~5 200 € |
| Yvelines | 78 | ~4 600 € |
| Seine-Saint-Denis | 93 | ~3 900 € |
| Essonne | 91 | ~3 000 € |
| Val-d’Oise | 95 | ~3 700 € |
| Seine-et-Marne | 77 | ~3 200 € |

*(Sources : MeilleursAgents, PAP, IMOP — 2024-2025)*

---

### 🧩 Valeur ajoutée pour le persona
> Ce widget offre à Marc Dubois une **vision immédiate et contextualisée** du marché foncier  
> et lui permet de cibler les **départements présentant le meilleur potentiel de rentabilité**  
> selon ses critères de budget et de type de bien.

---


In [1]:
%run widget_1.py

afficher_widget_prix_m2(luminosite="sombre")

C:\Users\Gobtrie\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Output()

---

## 🧩 Widget 2 — Évolution temporelle du prix au m² (2019 – 2024)

### 🎯 Objectif
Ce widget permet à **Marc Dubois**, investisseur spécialisé dans la réhabilitation de biens anciens,  
d’analyser **l’évolution du prix médian au m² sur les 5 dernières années**.  
L’objectif est d’identifier les **communes en gentrification** (forte revalorisation)  
et les **zones en stagnation ou en baisse** afin d’orienter ses investissements vers  
les marchés les plus dynamiques et prometteurs.

---

### ⚙️ Données utilisées
Fichier : `data/dvf_idf_2019_2024_clean.csv`  
Colonnes exploitées :
- `annee` → Année de mutation (2019 à 2024)  
- `code_commune` → Code INSEE de la commune  
- `code_departement` → Département du bien (75 à 95)  
- `type_local` → Type de bien (*Maison* / *Appartement*)  
- `prix_m2` → Prix médian au m² (calculé à partir du DVF)

---

### 🧰 Paramètres interactifs
| Paramètre | Type | Description |
|------------|------|-------------|
| **Département** | Liste déroulante | Sélection du département d’analyse |
| **Commune(s)** | Sélecteur multiple | Choix d’une ou plusieurs communes à comparer |
| **Type de bien** | Liste déroulante | Filtre entre *Maisons* et *Appartements* |
| **Afficher tendance** | Case à cocher | Ajoute une droite de régression linéaire pour visualiser la tendance |

---

### 📊 Sortie du widget
- **Graphique en courbes** montrant l’évolution du **prix médian au m²** (2019 → 2024)  
  - Une ligne par commune sélectionnée  
  - Option d’affichage d’une **tendance linéaire** pour chaque courbe  
  - Couleurs et styles adaptés automatiquement au mode **clair / sombre**
- **Tableau récapitulatif** indiquant :
  - Le prix médian en 2019 et 2024  
  - La **variation en pourcentage (%)** sur la période  
  - Un classement automatique des communes selon la plus forte croissance

---

### 🧠 Interprétation attendue
Ce widget aide Marc à :
- Déterminer les **zones en revalorisation rapide** (potentiel de plus-value)  
- Éviter les **zones à stagnation ou décroissance** du marché  
- Comparer facilement plusieurs communes d’un même département  
- Identifier les **tendances structurelles** à moyen terme

---

### 💬 Exemple d’interprétation
> Entre 2019 et 2024, les prix médians au m² ont augmenté de **+24 % à Montreuil (93)**  
> tandis qu’ils n’ont progressé que de **+4 % à Melun (77)**.  
> Marc peut ainsi prioriser ses investissements dans des communes où la dynamique  
> du marché est confirmée par la tendance.

---

### 🧩 Valeur ajoutée pour le persona
> Cet outil donne à Marc Dubois une **vision temporelle claire et fiable du marché immobilier**.  
> Il lui permet de **repérer les zones à forte plus-value**, de **mesurer la dynamique locale**  
> et d’**appuyer ses choix d’achat-rénovation sur des tendances réelles et mesurées**.  

---


In [ ]:
%run widget_2.py

afficher_widget_evolution_prix(luminosite="sombre")

---

## 🧩 Widget 3 — Carte de chaleur du prix au m² (Heatmap géographique)

### 🎯 Objectif

Ce widget permet à Marc DUBOIS, analyste immobilier,
de visualiser la répartition spatiale du prix au m² sur une carte interactive.
L’objectif est d’identifier les zones les plus chères, les secteurs abordables
et les opportunités d’investissement en fonction de la localisation,
du type de bien et du département sélectionné.

In [7]:
%run widget_3.py
widget_carte = CarteChaleursImmobilier('data/donnees_immobilieres.csv')
widget_carte.afficher_widget()

✅ Données chargées (50,000 lignes)


---

## 🧩 Widget 4 : Alan

### 🎯 Objectif


In [17]:
%run widget_4.py

---

## 🧩 Widget 5 : Ayoub

### 🎯 Objectif


---

## 🧩 Widget 6 : Ayoub

### 🎯 Objectif
